In [ ]:
import ee
import geemap

In [ ]:
map =geemap.Map()
map

In [ ]:
landsat7 = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003').select([0,1,2,3,4,6])
landsat_vis = {'bands': ['B4', 'B3', 'B2'], 'gamma': 1.4}
map.addLayer(landsat7, landsat_vis, "LE7_TOA_5YEAR/1999_2003")

hyperion = ee.ImageCollection('EO1/HYPERION').filter(
    ee.Filter.date('2015-01-01', '2015-03-01')
)
hyperion_vis = {
    'min': 1000.0,
    'max': 14000.0,
    'gamma': 2.5,
}
map.addLayer(hyperion, hyperion_vis, 'EO1/HYPERION');

In [ ]:
map.set_plot_options(add_marker_cluster=True, overlay=None)

In [ ]:
m.plot_demo()

In [ ]:
## Adding shapefile
tn_shp = 'data/trentino_shp.shp'
shapefile = geemap.shp_to_ee(tn_shp)
map.addLayer(shapefile, {}, 'TN_Shapefile')

In [ ]:
## export shapefile as kml
geemap.ee_export_vector(shapefile, filename= 'data/trentino_shp.kml')

In [ ]:
## split map window
map =geemap.Map()
map.split_map(left_layer='ESA WorldCover 2020', right_layer='ESA WorldCover 2021')
map

In [ ]:
basemaps = geemap.basemaps.keys()
print(basemaps)

In [ ]:
## Earth engine dataset

img_collection = ee.ImageCollection("COPERNICUS/Landcover/100m/Proba-V-C3/Global")
print(img_collection.aggregate_array('system:id').getInfo())

In [ ]:
aafc_2009 = ee.Image('AAFC/ACI/2009').select('landcover')
aafc_2020 = ee.Image('AAFC/ACI/2020').select('landcover')

left_layer = geemap.ee_tile_layer(aafc_2009, {}, 'AAFC 2009')
right_layer = geemap.ee_tile_layer(aafc_2020, {}, 'AAFC 2020')

map = geemap.Map()
map.split_map(left_layer, right_layer)
map

In [ ]:
## clip ee image to shapefile
map = geemap.Map()
map

In [ ]:

image = ee.Image('USGS/SRTMGL1_003')
# Set visualization parameters.
vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}

# Add Earth Engine DEM to map
map.addLayer(image, vis_params, 'SRTM DEM')

states = 'data/trentino_shp.shp'
states_shp = geemap.shp_to_ee(tn_shp)
map.addLayer(states_shp, {}, 'US States')

In [ ]:
roi = image.clip(states_shp)
map.addLayer(roi, vis_params, 'Clipped image')

In [ ]:
clipped_image = image.clip(roi)
map.addLayer(clipped_image, vis_params, 'Clipped image')